# Нейронные сети классификации изображений

# 3 Разветвленные сети, сеть NiN
Сети LeNet, AlexNet, и VGG работают по одному шаблону:
- извлекаем признаки из пространственной структуры данных, т.е. связываем ближайшие пиксели.
- полносвязная сеть типа многослойного персептрона классифицирует на основе таких признаков. При это в персептроне теряется пространственная информация, связываются *все* признаки.

Логичен вопрос, а почему бы не использовать полносвязные слои раньше, не только в конце. Если это делать обычным способом, то потеряется пространственная информация, которая для изображений может быть важна.

Нужно и не потерять пространственную информацию, и обработать признаки более сложным образом, чем просто свертка. Ответ - связывать только каналы.

Для этого подойдет свертка с ядром 1 на 1. Действительно, что делает такая **многоканальная** свертка? Берет входной признак с первого канала, умножает на единственный вес в первом ядре, берет признак второго канала, умножает его на вес второго ядра, и т.д., затем складывает результаты от всех каналов и добавляет смещение.

Абсолютно то же самое, что делает персептронный нейрон. Но для каналов, а не всего изображения целиком.

Получилась разветвленная архитектура, где каждый пиксель (признак) обрабатывается независимо от остальных. Как бы одна сеть - персептрон - применяется много раз для признаков другой сети, сеть в сети.



## NiN блоки
Такую сеть так и назвали Network in Network (NiN), состоит из блоков.

Первый слой в блоке - сверточный, с заданным размером ядра, связывает локально входные признаки, каждый фильтр дает свою карту признаков.

Следующие слои (в наших примерах два) связывают каналы признаков независимо для каждого признака, не меняют пространственную форму. Разумеется фильтров в таких слоях также несколько. Технически реализуются как свертка с ядром 1 на 1.    

На рис. показана архитектура блоков и самой сети NiN.

![img](https://d2l.ai/_images/nin.svg)

Между блоками используют слои субдискретизации. Отметим, что последний слой субдискретизации это слой 2D Global average pooling, который находит среднее значение для каждого канала входа.

![img](https://qph.cf2.quoracdn.net/main-qimg-a8c5cb77e38e589b3d7744c60ab5daad-pjlq)

Таким образом в сети **нет полносвязных слоев** в обычном понимании, на которые, как мы видели, приходятся основные вычислительные затраты. Число фильтров последнего блока определяется числом классов.



In [1]:
# установим вспомогательные функции из курса  d2l
!pip -q install d2l==0.17.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


Зададим блоки NiN. Принимает аргументы:
- num_channels - число каналов (фильтров)
- kernel_size - размер сверточного слоя,
- strides - сдвиг для сверточного слоя,
- padding - набивка для него же.

Несмотря на разветвленную архитектуру все еще можем реализовать такую сеть в контейнере Sequential.

In [2]:
import tensorflow as tf
from tensorflow.keras import datasets

def nin_block(num_channels, kernel_size, strides, padding):
    return tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(num_channels, kernel_size, strides=strides,
                               padding=padding, activation='relu'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                               activation='relu'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                               activation='relu'),
        tf.keras.layers.Conv2D(num_channels, kernel_size=1,
                               activation='relu')])

In [3]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

## Сеть NiN

Оригинальная сеть NiN использует сверточные слои в блоках с ядрами $11\times 11$, $5\times 5$, и $3\times 3$, такое же число фильтров, что и в AlexNet.

После блока NiN используют слой maxipooling со сдвигом 2 и ядром $3\times 3$.

В число каналов последнего блока совпадает с числом классов и используется слой *global average pooling*.



In [12]:
# Сеть NiN
def net():
    return tf.keras.models.Sequential([
        nin_block(384, kernel_size=7, strides=3, padding='same'), # блок 1
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        nin_block(756, kernel_size=5, strides=1, padding='same'), # блок 2
        tf.keras.layers.MaxPool2D(pool_size=3, strides=2),
        nin_block(1024, kernel_size=3, strides=1, padding='same'), # блок 3
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),
        tf.keras.layers.Dropout(0.5),
        # There are 10 label classes
        nin_block(20, kernel_size=3, strides=1, padding='same'), # блок 4, число фильтров равно числу классов
        tf.keras.layers.GlobalAveragePooling2D()])

Проверим размерности выходов всех слоев. Первое измерение - примеры.

Посчитайте эти размерности самостоятельно, затем объясните вывод.



In [13]:
X = tf.random.uniform((1, 32*32, 32*32, 3))
for layer in net().layers:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 (1, 342, 342, 384)
MaxPooling2D output shape:	 (1, 170, 170, 384)
Sequential output shape:	 (1, 170, 170, 756)
MaxPooling2D output shape:	 (1, 84, 84, 756)
Sequential output shape:	 (1, 84, 84, 1024)
MaxPooling2D output shape:	 (1, 42, 42, 1024)
Dropout output shape:	 (1, 42, 42, 1024)
Sequential output shape:	 (1, 42, 42, 20)
GlobalAveragePooling2D output shape:	 (1, 20)


In [6]:
len(train_images[0][0])

32

In [7]:
input_shape=(128,32*32, 32*32, 3)
nn=net()
nn.build(input_shape)
nn.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_5 (Sequential)            │ (128, 342, 342, 96)         │          42,144 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (128, 170, 170, 96)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_6 (Sequential)            │ (128, 170, 170, 256)        │         812,032 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (128, 84, 84, 256)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_7 (Sequential)            │ (128, 84, 84, 384)          │       1,328,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (128, 42, 42, 384)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (128, 42, 42, 384)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ sequential_8 (Sequential)            │ (128, 42, 42, 20)           │          70,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (128, 20)                   │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,253,216 (8.60 MB)

 Trainable params: 2,253,216 (8.60 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
import math

def cosine_annealing_lr(epoch, lr, num_epochs):
    return initial_lr * (1 + math.cos(math.pi * epoch / num_epochs)) / 1.8

In [9]:
initial_lr = 5e-4
num_epochs = 50
optimizer = tf.keras.optimizers.AdamW(learning_rate = initial_lr)
nn.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: cosine_annealing_lr(epoch, initial_lr, num_epochs))

In [11]:
%%time
history = nn.fit(train_images, train_labels, epochs=num_epochs,
                    validation_data=(test_images, test_labels),
                    batch_size = 128,  callbacks=[lr_callback])

Epoch 1/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - accuracy: 0.1030 - loss: 2.7838 - val_accuracy: 0.1857 - val_loss: 2.4655 - learning_rate: 5.5556e-04
Epoch 2/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.2092 - loss: 2.4122 - val_accuracy: 0.3192 - val_loss: 2.1089 - learning_rate: 5.5501e-04
Epoch 3/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - accuracy: 0.3618 - loss: 2.0006 - val_accuracy: 0.4072 - val_loss: 1.9015 - learning_rate: 5.5337e-04
Epoch 4/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.4305 - loss: 1.8265 - val_accuracy: 0.4440 - val_loss: 1.8159 - learning_rate: 5.5064e-04
Epoch 5/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.4663 - loss: 1.7267 - val_accuracy: 0.4914 - val_loss: 1.6758 - learning_rate: 5.4683e-04
Epoch 6/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5003 - loss: 1.6263 - val_accuracy: 0.4899 - val_loss: 1.6679 - learning_rate: 5.4196e-04
Epoch 7/50
391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step -

In [10]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

predictions = net.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)

# Создание матрицы ошибок
cm = confusion_matrix(test_labels, predicted_labels)

# Визуализация матрицы ошибок
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(test_labels))
disp.plot(cmap=plt.cm.Blues)
plt.show()

## Задания

- Настройте гиперпараметры для большей аккуратности решения.
- Почему в блоке два слоя 1 на 1? Удалите и или добавьте еще один, как это отразится на результате?
- Посчитайте производительность NiN:
    - Сколько обучаемых параметров?
    - Сколько вычислительных операций (умножения)?
    - Объем памяти необходимый для сети?



# Ссылки
Использованы и адаптированы материалы

https://d2l.ai/chapter_convolutional-modern/nin.html